# Properties and Descriptor

In [1]:
from numbers import Integral

In [2]:
class Person:
    @property
    def age(self):
        print("getter called")
        return getattr(self,"_age",None)
    @age.setter
    def age(self,value):
        print("setter called")
        if not isinstance(value,Integral):
            raise ValueError(f"age: must be integer")
        elif value < 0:
            raise  ValueError(f"age cannot be non negative integer")
        self._age = value

In [3]:
p = Person()

In [4]:
try:
    p.age = -10
except ValueError as ex:
    print(ex)

setter called
age cannot be non negative integer


In [5]:
p.age =10

setter called


In [6]:
p.__dict__

{'_age': 10}

In [7]:
p.__dict__["age"] = "hello"

In [8]:
p.__dict__

{'_age': 10, 'age': 'hello'}

In [9]:
p.age

getter called


10

1. same behavior as we have seen in the data descriptor.
2. class attribute not shadow by instance attribute
3. So instead so storing in the different name we can use the same name.

We cant use the same name for the non data descriptor.

In [10]:
from numbers import Integral
class Person:
    @property
    def age(self):
        print("getter called")
        return self.__dict__.get("age",None)

    @age.setter
    def age(self,value):
        print("setter called")
        if not isinstance(value,Integral):
            raise ValueError(f"age: must be integer")
        elif value < 0:
            raise  ValueError(f"age cannot be non negative integer")
        self.__dict__["age"] = value

In [11]:
p = Person()

In [12]:
p.age = 10

setter called


In [13]:
p.age

getter called


10

In [14]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'age': <property at 0x24dbbd26700>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [15]:
p.__dict__

{'age': 10}

# Not using as decorator

In [16]:
from numbers import Integral
class Person:
    def get_age(self):
        print("getter called")
        return self.__dict__.get("age",None)

    def set_age(self,value):
        print("setter called")
        if not isinstance(value,Integral):
            raise ValueError(f"age: must be integer")
        elif value < 0:
            raise  ValueError(f"age cannot be non negative integer")
        self.__dict__["age"] = value

    age = property(fget=get_age,fset=set_age)

In [17]:
p = Person()

In [18]:
import operator

In [19]:
check_descriptor = operator.attrgetter("__set__","__get__","__delete__")

In [20]:
check_descriptor(Person.age)

(<method-wrapper '__set__' of property object at 0x0000024DBBD24130>,
 <method-wrapper '__get__' of property object at 0x0000024DBBD24130>,
 <method-wrapper '__delete__' of property object at 0x0000024DBBD24130>)

we can implement our own version of property decorator

In [21]:
class Test:
    @property
    def test(self):
        pass

In [22]:
t = Test()
try:
    t.test =100
except AttributeError as ex:
    print(ex)

property 'test' of 'Test' object has no setter


In [23]:
class Test:
    def set_test(self,value):
        pass
    test = property(fset=set_test)


t = Test()
t.test =100

In [24]:
try:
    t.test
except AttributeError as ex:
    print(ex)

property 'test' of 'Test' object has no getter


In [25]:
class MakeProperty:
    def __init__(self,fget=None,fset=None):
        self.fget = fget
        self.fset = fset
    def __set_name__(self, owner, name):
        self.prop_name = name

    def __get__(self, instance, owner):
        print("property get called... ")
        if instance is None:
            return self
        elif self.fget is None:
            raise AttributeError(f"property '{self.prop_name}' of '{instance.__class__.__name__}' object has no getter ")
        return self.fget(instance)

    def __set__(self, instance, value):
        print("property set called... ")
        if self.fset is None:
            raise AttributeError(f"property '{self.prop_name}' of '{instance.__class__.__name__}' object has no setter ")
        self.fset(instance,value)


In [26]:
from numbers import Integral
class Person:
    def get_age(self):
        print("getter called")
        return self.__dict__.get("age",None)

    def set_age(self,value):
        print("setter called")
        if not isinstance(value,Integral):
            raise ValueError(f"age: must be integer")
        elif value < 0:
            raise  ValueError(f"age cannot be non negative integer")
        self.__dict__["age"] = value

    age = MakeProperty(fget=get_age,fset=set_age)

In [27]:
p = Person()

In [28]:
try:
    p.age = -10
except ValueError as ex:
    print(ex)

property set called... 
setter called
age cannot be non negative integer


In [29]:
p.age =10

property set called... 
setter called


In [30]:
p.age

property get called... 
getter called


10

In [31]:
from numbers import Integral
class Person:
    def get_age(self):
        print("getter called")
        return self.__dict__.get("age",None)

    def set_age(self,value):
        print("setter called")
        if not isinstance(value,Integral):
            raise ValueError(f"age: must be integer")
        elif value < 0:
            raise  ValueError(f"age cannot be non negative integer")
        self.__dict__["age"] = value

    age = MakeProperty(fget=get_age)

In [32]:
p = Person()

In [33]:
try:
    p.age = 10
except AttributeError as ex:
    print(ex)

property set called... 
property 'age' of 'Person' object has no setter 


In [34]:
from numbers import Integral
class Person:
    def get_age(self):
        print("getter called")
        return self.__dict__.get("age",None)

    def set_age(self,value):
        print("setter called")
        if not isinstance(value,Integral):
            raise ValueError(f"age: must be integer")
        elif value < 0:
            raise  ValueError(f"age cannot be non negative integer")
        self.__dict__["age"] = value

    age = MakeProperty(fset=set_age)

In [35]:
p =Person()
try:
    p.age
except AttributeError as ex:
    print(ex)

property get called... 
property 'age' of 'Person' object has no getter 


## As decorator

In [36]:
class MakeProperty:
    def __init__(self,fget=None,fset=None):
        self.fget = fget
        self.fset = fset
    def __set_name__(self, owner, name):
        self.prop_name = name

    def __get__(self, instance, owner):
        print("property get called... ")
        if instance is None:
            return self
        elif self.fget is None:
            raise AttributeError(f"property '{self.prop_name}' of '{instance.__class__.__name__}' object has no getter ")
        return self.fget(instance)

    def __set__(self, instance, value):
        print("property set called... ")
        if self.fset is None:
            raise AttributeError(f"property '{self.prop_name}' of '{instance.__class__.__name__}' object has no setter ")
        self.fset(instance,value)

    def setter(self,fset):
        self.fset = fset
        return self


In [37]:
from numbers import Integral
class Person:
    @MakeProperty
    def age(self):
        print("getter called")
        return self.__dict__.get("age",None)

    @age.setter
    def age(self,value):
        print("setter called")
        if not isinstance(value,Integral):
            raise ValueError(f"age: must be integer")
        elif value < 0:
            raise  ValueError(f"age cannot be non negative integer")
        self.__dict__["age"] = value

In [38]:
p = Person()

In [39]:
try:
    p.age = -10
except ValueError as ex:
    print(ex)

property set called... 
setter called
age cannot be non negative integer


In [40]:
p.age = 10

property set called... 
setter called


In [41]:
p.__dict__

{'age': 10}

In [42]:
p.age

property get called... 
getter called


10